## IEEE Xplore Simple Request Example

Example of a simple request searching for the combination Machine Learning AND (boolean) coronary in the configured metadata fields and abstracts of the articles in the database.

In [13]:
import requests

apikey = 'apikey=8m22c725rj99bvxq6pftexqk'
address = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?'
querytext = '&meta_data=((Machine%20Learning)%20AND%20coronary)'
maxrecords = "&max_records=200"

url = address + apikey + querytext + maxrecords

r = requests.get(url)

print("request: \n", r.url, "\n")
print("keys \n", r.json().keys())
print("Total Records:", r.json().get('total_records'))
print("Total Received:",len(r.json().get('articles')))


request: 
 http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=8m22c725rj99bvxq6pftexqk&meta_data=((Machine%20Learning)%20AND%20coronary)&max_records=200 

keys 
 dict_keys(['articles', 'total_records', 'total_searched'])
Total Records: 66
Total Received: 66


## IEEE_Xplore_Searcher class

Articles search class for IEEE Explore database using your api. 

In [9]:
import requests

class IEEE_Xplore_Searcher():
    apikey = '8m22c725rj99bvxq6pftexqk'
    address = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?'
    
    def __init__(self, queryterms: list=None, apikey: str=None, address: str=None): 
        self.articles_found = []
        if queryterms: 
            self.queryterms = queryterms
        if apikey: 
            self.apikey = apikey
        if address: 
            self.address = address
            
    def search(self, queryterms: list=None, search_type: str="meta_data",
                    start_year: int=None, end_year: int=None,
                    content_type: str=None, start_record: int=1, 
                    sort_field: str=None, sort_order: int=None, 
                    max_records: int=200, article_title: str=None,
                    author: str=None):
        """
        @param queryterms: list of lists. Terms within the same list are
            separated by an OR. Lists are separated by an AND
        @param search_type: meta_data or querytext. 
            meta_data: This field enables a free-text search of all 
                configured metadata fields and the abstract. Accepts 
                complex queries involving field names and  boolean 
                operators.
            querytext: This field enables a free-text search of all 
                configured metadata fields, abstract and document text. 
                Accepts complex queries involving field names and boolean 
                operators. 
        @param start_year: Start value of Publication Year to restrict results by.
        @param end_year: End value of Publication Year to restrict results by.
        @param content_type: Note: these are case sensitive and must be spelled as 
            presented here to get a result: Journals, Conference, Early Access,
            Standards, Books, Courses
        @param start_record: Sequence number of first record to fetch. Default: 1
        @param sort_field: Field name on which to sort. Choose from: article_number
            article_title, author, publication_title, publication_year
        @param sort_order: asc (for ascending sort) or desc (for descending sort)
        @param max_records: The number of records to fetch. Maximum: 200
        @param article_title: Title of an individual document (journal article, 
            conference paper, standard, eBook chapter, or course).
        @param author: An author's name. Searches both first name and last name
        
        @return  the data fields returned by the search are described by the 
            following link https://developer.ieee.org/docs/read/Metadata_API_responses
        """
        
        if not queryterms: 
            queryterms = self.queryterms
        
        formated_query = " AND ".join(["(%s)" % " OR ".join(term)for term in queryterms])

        url = self.address + 'apikey=' + self.apikey  
        if search_type == "meta_data":
            url += '&meta_data=' + formated_query
        elif search_type == "querytext": 
            url += '&querytext=' + formated_query

        if start_year: 
            url += '&start_year=' + str(start_year)
        if end_year: 
            url += '&end_year=' + str(end_year)
        if content_type: 
            url += '&content_type=' + content_type
        if start_record: 
            url += '&start_record=' + str(start_record)
        if sort_field: 
            url += '&sort_field=' + sort_field 
        if sort_order: 
            url += '&sort_order=' + sort_order
        if max_records: 
            url += '&max_records=' + str(max_records)
        if article_title: 
            url += '&article_title=' + article_title
        if author: 
            url += '&author=' + author
        
        r = requests.get(url)
        print("Requisição: ")
        print("\n", r.url, "\n")
        total_records = r.json().get('total_records')
        print(round((total_records/max_records)+0.4999), " requests needed...")
        self.articles_found = requests.get(url).json().get('articles')
        print("a request completed...")
        while (start_record + max_records) < total_records:
            start_record += max_records
            url = url.replace('&start_record='+str(start_record-max_records), 
                              '&start_record='+str(start_record) )
            self.articles_found += requests.get(url).json().get('articles')
            print("a request completed...")
            
        while len(self.articles_found) < total_records:
            pass
        
        return self.articles_found
    

## Using the IEEE_Xplore_Searcher class

In [11]:

technology_queryterms = [
    'machine learning', 'deep learning', 'artificial intelligence', 
    'neural network', 'scoring system'
]

health_queryterms = [
    'coronary artery disease', 'chest pain', 'heart disease', 'MACE', 
    'Acute Cardiac Complications'
]

queryterms = [technology_queryterms, health_queryterms]

ieee_searcher = IEEE_Xplore_Searcher()
articles = ieee_searcher.search(queryterms=queryterms,
                  start_year=1975, content_type="Journals", 
                  search_type="meta_data", max_records=200)  

print("\nTotal found:", len(articles))
print("\nArticle fields\n", articles[0].keys())
print("\nArticle List\n")
for i, article in enumerate(articles): 
    print(i+1, " - ", article.get('title'))

Requisição: 

 http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=8m22c725rj99bvxq6pftexqk&meta_data=(machine%20learning%20OR%20deep%20learning%20OR%20artificial%20intelligence%20OR%20neural%20network%20OR%20scoring%20system)%20AND%20(coronary%20artery%20disease%20OR%20chest%20pain%20OR%20heart%20disease%20OR%20MACE%20OR%20Acute%20Cardiac%20Complications)&start_year=1975&content_type=Journals&start_record=1&max_records=200 

1  requests needed...
a request completed...

Total found: 118

Article fields
 dict_keys(['publication_number', 'index_terms', 'rank', 'pdf_url', 'publication_title', 'doi', 'volume', 'content_type', 'issn', 'abstract_url', 'citing_paper_count', 'title', 'publication_date', 'issue', 'end_page', 'authors', 'is_number', 'partnum', 'abstract', 'article_number', 'start_page', 'publisher', 'access_type'])

Article List

1  -  Detecting Cardiovascular Disease from Mammograms With Deep Learning
2  -  Machine Learning Techniques as a Helpful Tool Toward Determina